In [10]:
#%matplotlib qt
from keras.models import Sequential, load_model
from keras.datasets import imdb
from keras.layers import Dense, Dropout
from keras.layers import LSTM, RNN, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GRU
from keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
#import matplotlib.pyplot as plt

ImportError: No module named matplotlib

In [3]:
pos_set = np.load("./train/pos.npy")
neg_set = np.load("./train/neg.npy")

complete = np.concatenate((pos_set, neg_set))
y = np.concatenate((np.ones((pos_set.shape[0],1)), np.zeros((neg_set.shape[0],1))))

scaled_data = []

for trajectory in complete:
    std = StandardScaler()
    scaled_data.append(std.fit_transform(trajectory))

scaled_data = np.asarray(scaled_data)

In [4]:
seed = 42
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
histories = []
for train, test in kfold.split(scaled_data, y):
  # create model
    print("making model")
    
    model_cnn = Sequential()
    model_cnn.add(Conv1D(128, 5, activation="relu", input_shape=(999,2)))
    model_cnn.add(MaxPooling1D())
    model_cnn.add(Conv1D(64, 5, activation="relu"))
    model_cnn.add(MaxPooling1D())
    model_cnn.add(Conv1D(32, 5, activation="relu"))
    model_cnn.add(GlobalMaxPooling1D())
    model_cnn.add(Dense(1, activation="sigmoid"))

    # Compile model
    
    model_cnn.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    # Fit the model
    print("training model")
    history = model_cnn.fit(scaled_data[train], y[train], epochs=4)
    # evaluate the model
    print("evaluating model")
    scores = model_cnn.evaluate(scaled_data[test], y[test], verbose=0)
    print("%s: %.2f%%" % (model_cnn.metrics_names[1], scores[1]*100))
    print("training accuracy: %.2f" %(history.history["acc"])[1])
    cvscores.append(scores[1] * 100)
    histories.append(history.history)
    print("\n\n")
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

making model
training model
Epoch 1/4
923/923 [==============================] - 11s 12ms/step - loss: 0.7098 - acc: 0.5103
Epoch 2/4
923/923 [==============================] - 10s 11ms/step - loss: 0.6931 - acc: 0.5200
Epoch 3/4
923/923 [==============================] - 10s 10ms/step - loss: 0.6926 - acc: 0.5395
Epoch 4/4
923/923 [==============================] - 10s 11ms/step - loss: 0.6917 - acc: 0.5179
evaluating model
acc: 44.66%
training accuracy: 0.52



making model
training model
Epoch 1/4
923/923 [==============================] - 10s 11ms/step - loss: 0.7137 - acc: 0.4897
Epoch 2/4
923/923 [==============================] - 10s 10ms/step - loss: 0.6949 - acc: 0.5211
Epoch 3/4
923/923 [==============================] - 10s 11ms/step - loss: 0.6984 - acc: 0.5016
Epoch 4/4
923/923 [==============================] - 10s 11ms/step - loss: 0.6905 - acc: 0.5320
evaluating model
acc: 47.57%
training accuracy: 0.52



making model
training model
Epoch 1/4
923/923 [=================

In [5]:
model_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_28 (Conv1D)           (None, 995, 128)          1408      
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 497, 128)          0         
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 493, 64)           41024     
_________________________________________________________________
max_pooling1d_20 (MaxPooling (None, 246, 64)           0         
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 242, 32)           10272     
_________________________________________________________________
global_max_pooling1d_10 (Glo (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 33        
Total para

In [14]:
np.save("./histories", histories)

In [17]:
c = np.load("./histories.npy")

In [19]:
c[0]

{'acc': [0.5102925244416086,
  0.5200433377196704,
  0.5395449626613671,
  0.5178764898366298],
 'loss': [0.7097995196623508,
  0.6930639732826828,
  0.6925926484885996,
  0.6916831139624442]}